# Scraping
#### Se realiza la carga de datos desde el sitio argenprop.com 

### Intalación de las librerías necesarias

In [1]:
pip install tqdm

    100% |████████████████████████████████| 81kB 1.2MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install bs4

    100% |████████████████████████████████| 102kB 781kB/s a 0:00:01
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /root/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

    100% |████████████████████████████████| 5.5MB 292kB/s ta 0:00:011
Note: you may need to restart the kernel to use updated packages.


### Importamos las librerías y creamos el dataset de Pandas

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm.auto import tqdm
import os

df=pd.DataFrame(columns=['Precio','Ubicacion','Barrio','Metros','Dorm','Baños','Años'])

### Obtenemos el número total de páginas de publicaciones

In [5]:
payload = {'api_key': 'e2cfbc1810d6a0321cfc04c45c315c05', 'url': 'https://www.argenprop.com/departamento-venta-region-capital-federal-1-o-más-cocheras-pagina-2'}
r = requests.get('http://api.scraperapi.com', params=payload)
soup = BeautifulSoup(r.text, 'lxml')
paginas = soup.find_all('li', class_="pagination__page")
paginas = int(paginas[-2].text)
print("Total de páginas:",paginas)

Total de páginas: 858


### Recolectamos la información de cada página, la parseamos y la incuimos en el dataset

In [7]:

for j in tqdm(range(2,10)): # PARA MENOR TIEMPO reemplazar paginas por otro valor así no se trae la totalidad de páginas 
    payload = {'api_key': 'e2cfbc1810d6a0321cfc04c45c315c05', 'url': 'https://www.argenprop.com/departamento-venta-region-capital-federal-1-o-más-cocheras-pagina-'+str(j)}
    r = requests.get('http://api.scraperapi.com', params=payload)
    
    soup = BeautifulSoup(r.text, 'lxml')

    precio = soup.find_all('p',class_="card__price")
    caract = soup.find_all('ul',class_="card__main-features")
    direccion = soup.find_all('p',class_="card__title--primary")

    try:
        ind = df.index[-1]+1
    except: 
        ind = 0

    for i in range(len(precio)):

        precios = precio[i].text.replace("USD ","").replace("$ ","").split()
        caracts = caract[i].text.split()
        direcciones = direccion[i].text.replace("Departamento en Venta en ","").split(", ")
        
        df.loc[ind+i,'Ubicacion'] = direcciones[0]
        df.loc[ind+i,'Barrio'] = direcciones[1]
        
        if precios[0] == 'Consultar' :
            df.loc[ind+i,'Precio'] = 999
        else:
            df.loc[ind+i,'Precio'] = int(precios[0].replace('.',''))

        if 'm²' in caracts:
            df.loc[ind+i,'Metros'] = float(caracts[caracts.index('m²')-1].replace(',','.'))

        if 'baño' in caracts:
            df.loc[ind+i,'Baños'] = int(caracts[caracts.index('baño')-1])

        if 'baños' in caracts:
            df.loc[ind+i,'Baños'] = int(caracts[caracts.index('baños')-1])

        if 'Monoam.' in caracts:
            df.loc[ind+i,'Dorm'] = 0
    
        if 'dorm.' in caracts:
            df.loc[ind+i,'Dorm'] = int(caracts[caracts.index('dorm.')-1])

        if 'años' in caracts:
            df.loc[ind+i,'Años'] = int(caracts[caracts.index('años')-1])

        if 'Estrenar' in caracts:
            df.loc[ind+i,'Años'] = 0
        

  0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
import findspark

findspark.add_jars('/app/postgresql-42.1.4.jar')
findspark.init()

In [9]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("pyspark-postgres")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

### Convertimos los valores faltantes en '999' y luego transformamos el Dataframe de Pandas en Spark

In [10]:
df=df.fillna(999)
df = spark.createDataFrame(df)

### Guardamos en Postgres

In [11]:
df \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/workshop") \
    .option("dbtable", "workshop.viviendas") \
    .option("user", "workshop") \
    .option("password", "w0rkzh0p") \
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite') \
    .save()

In [12]:
df_pos = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/workshop") \
    .option("dbtable", "workshop.viviendas") \
    .option("user", "workshop") \
    .option("password", "w0rkzh0p") \
    .option("driver", "org.postgresql.Driver") \
    .load()

### Corroboramos que coincidan la cantidad de filas y columnas y mostramos las primeras 100 filas del Dataframe importado de postgres

In [13]:
print((df.count(), len(df.columns)))
print((df_pos.count(), len(df_pos.columns)))

df_pos.show(100)

(120, 7)
(120, 7)
+-------+-----------------+---------------+------+----+-----+----+
| Precio|        Ubicacion|         Barrio|Metros|Dorm|Baños|Años|
+-------+-----------------+---------------+------+----+-----+----+
| 230000|         Belgrano|Capital Federal|  95.0|   3|    3|  45|
| 145000|           Flores|Capital Federal|  50.0|   1|    1|   2|
|1350000|    Palermo Chico|        Palermo| 228.0|   5|    5|  40|
| 300000|     Villa Devoto|Capital Federal| 103.0|   3|    3|  35|
| 120000|         Belgrano|Capital Federal|  70.0|   2|    1|  48|
| 162000|       Belgrano R|       Belgrano|  67.0|   2|    1|  35|
| 440000|        Caballito|Capital Federal| 148.0|   3|    2|  31|
|  90000|      San Nicolás|         Centro|  33.0|   0|    1| 999|
| 172000|           Retiro|         Centro|  75.0|   2|    2| 999|
|1500000|    Puerto Madero|         Centro| 230.0|   3|    4|  10|
| 155000|    Villa Urquiza|Capital Federal|  54.0|   2|    2|   0|
| 197000|            Nuñez|Capital Federal| 

In [ ]:
spark.stop()